In [ ]:
import stegoton
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation
import numpy as np

import stegoton_shapes  # Replace this with the name of your script

# Overview

In this notebook we investigate the variation in shape (as a function of amplitude) of stegotons.  LeVeque and Yong observed (by measuring the wave as a function of time at a fixed point in space) that stegotons have something close to a $sech^2$ shape, with their width scaling like the reciprocal square root of their amplitude.  In this way they are similar to KdV solitons.

The KdV equation 
$$
    u_t + \frac{1}{2} (u^2)_x + u_{xxx} = 0
$$
has a linear dispersive term, whereas the dispersive terms in the LY homogenized equations are nonlinear.  A simple PDE with nonlinear dispersion is the K(2,2) compacton equation
$$
    u_t + (u^2)_x + (u^2)_{xxx} = 0.
$$
The latter equation has solitons with a different shape:
$$
    u(x,t) = A \cos^2(B(x-Ct)).
$$
A connection for stegotons with the compacton equation was made in my PhD thesis from a theoretical standpoint.  From the homogenized equations of LeVeque & Yong one can see that the leading dispersive terms are a sum of linear and nonlinear parts.

It seems therefore unsurprising that stegotons of small amplitude have a nearly $sech^2$ shape, while large-amplitude stegotons have a $\cos^2$ (or mabye even steeper) shape.

# A train of solitary waves
First we generate a long train of solitary waves of descending amplitude.  One issue with investigating large-amplitude stegotons is that there seems to be a limit on how large they can get without leading to shock formation.  The initial pulse in the simulation below is just large enough that some ripples appear, probably caused by a little shock formation.

In [ ]:
claw = stegoton_shapes.setup(num_output_times=200,amp=0.17,cells_per_layer=40,width=30,tfinal=800.,solver_type="sharpclaw")
claw.keep_copy = True

claw.run()

## Rescale and re-center
We now find the peaks of the stegotons and then shift them all to $x=0$.  Then we rescale them to have amplitude 1, also rescaling their width.

In the code that follows we only plot the largest and smallest stegotons; plotting the others makes it hard to see the details.

In [ ]:
g = np.loadtxt("./_output/_gauges/gauge590.75.txt")
tt = g[:,0]
rhou = g[:,2]
rho = 4
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(620,795)
#plt.xlim(658,680)
delta = np.diff(-u)
dd = delta[1:]*delta[:-1] * (u[2:]<-0.1)
peaks = np.where(dd<0)
plt.plot(tt[peaks],-u[peaks],'ok')

In [ ]:
tt[peaks]

In [ ]:
centers = np.array([636.6, 752.86])# 643.29052281, 650.68345809, 658.6454708 ,
       #667.22, 676.37889282,
       #686.02734809, 696.2038154 ,
       #706.86557607, 717.89074234, 729.31871731, 741.01,
       #752.82, 764.5])#+0.05
for c in centers:
    plt.plot(tt-c,-u)
plt.plot([0,0],[0,2.85],"--")
plt.xlim(-5,5);

In [ ]:
-u[peaks]

In [ ]:
amplitudes = np.array([2.36945698,0.47904089])/rho#, 2.25992276, 2.13510977, 1.99824864, 1.84563516,
       #1.69427908, 1.52435368, 1.35967552,
       #1.17964447, 1.00611449, 0.83038067,
       #0.65329977, 0.47904089]
t = np.linspace(-2.5,2.5,1000)
plt.figure(figsize=(14,8))
j = 1
for (a,c) in zip(amplitudes,centers):
        plt.plot((tt-c)*np.sqrt(a),-u/a, label="stegoton"+str(j))
        j = j+1
plt.plot(t,1./(np.cosh(1.7*t))**2, 'k--', label="sech^2")
plt.plot(t,1./(np.cosh(1.4*t))**2, 'r--', label="sech^2")

plt.plot(t,(np.cos(1.3*t))**2,'g--', label="cos^2")
#plt.plot([0,0],[0,1.],"--")

plt.xlim(-1.5,1.5)
plt.ylim(-0.1,1.1);
plt.legend();
plt.plot([-1.5,1.5],[0,0],"--",color="gray")


Here we can see that the largest and smallest stegoton are shaped roughly like a $\cos^2$ and $sech^2$, respectively.  Although the tails (particularly for the larger one) differ markedly.

Above we have examined $u$.  What about $\sigma$?  It stays closer to a sech$^2$ shape:

In [ ]:
g = np.loadtxt("./_output/_gauges/gauge590.25.txt")
centers = np.array([636.075, 752.265])
amplitudes = np.array([1.53,0.255])
eps = g[:,1]
K = 1
sig = np.exp(K*eps)-1

plt.figure(figsize=(14,8))
j = 1
for (a,c) in zip(amplitudes,centers):
        plt.plot((tt-c)*np.sqrt(a),sig/a, label="stegoton"+str(j))
        j = j+1

plt.plot(t,1./(np.cosh(1.2*t))**2, 'k--', label="sech^2")
plt.plot(t,1./(np.cosh(1.*t))**2, 'k--', label="sech^2")
plt.plot(t,1./(np.cosh(0.43*t))**10, 'r--', label="sech^10")

plt.xlim(-2.5,2.5)
plt.ylim(-0.1,1.1)
#plt.plot(tt,sig)

In [ ]:
rho

# Generating individual large stegotons

An arbitrary initial pulse, like that used above, tends to break into multiple stegotons.  A larger initial pulse yields larger stegotons, but after a certain point we get shock formation and oscillations instead of taller stegotons.  It seems to be possible to generate taller stegotons by directly initializing with the appropriate shape, which is the strategy we try now.

In [ ]:
claw = stegoton_shapes.setup(num_output_times=200,amp=.1,cells_per_layer=40,width=10,tfinal=800.,
                             solver_type="sharpclaw",shape="cos2",xmax=200)
claw.keep_copy = True

claw.run()

In [ ]:
# amp = 0.1
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(705,730)

w = 0.45
c = 715.6
a = 0.099
plt.plot(tt,a*(np.cos(w*(tt-c)))**2 * (np.abs(tt-c)<=np.pi/(2*w)));

For amplitude 0.1, we see that the tails are substantially larger than those of the $\cos^2$ shape.

In [ ]:
claw = stegoton_shapes.setup(num_output_times=200,amp=.25,cells_per_layer=40,width=10,tfinal=800.,
                             solver_type="sharpclaw",shape="cos2",xmax=200)
claw.keep_copy = True

claw.run()

In [ ]:
# amp = 0.25
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(670,690)

w = 0.58
c = 683.1
a = 0.217
plt.plot(tt,a*(np.cos(w*(tt-c)))**2 * (np.abs(tt-c)<=np.pi/(2*w)));

For amplitude 0.25, they are closer.

In [ ]:
claw = stegoton_shapes.setup(num_output_times=200,amp=.42,cells_per_layer=40,width=10,tfinal=800.,
                             solver_type="sharpclaw",shape="cos2",xmax=200)
claw.keep_copy = True

claw.run()

In [ ]:
# amp = 0.42
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
#plt.xlim(620,630)

w = 0.7
c = 626.59
a = 0.425
plt.plot(tt,a*(np.cos(w*(tt-c)))**2 * (np.abs(tt-c)<=np.pi/(2*w)));

Here we see a nearly $\cos^2$ shape.

In [ ]:
claw = stegoton_shapes.setup(num_output_times=200,amp=.5,cells_per_layer=40,width=10,tfinal=800.,
                             solver_type="sharpclaw",shape="cos2",xmax=200)
claw.keep_copy = True

claw.run()

In [ ]:
# amp = 0.5
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(615,625)

w = 0.7
c = 619.75
a = 0.45
plt.plot(tt,a*(np.cos(w*(tt-c)))**2 * (np.abs(tt-c)<=np.pi/(2*w)));

p = 1.6
w = 0.8
cos = np.maximum(np.cos(w*(tt-c)),0)
fit = a*cos**p * (np.abs(tt-c)<=np.pi/(2*w))
#fit = np.real(fit)
plt.plot(tt,fit);

At even larger amplitudes, the stegoton tails are even steeper than $\cos^2$.

In [ ]:
# amp = 0.5
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(198,206)

w = 0.7
c = 201.8
a = 0.45
plt.plot(tt,a*(np.cos(w*(tt-c)))**2 * (np.abs(tt-c)<=np.pi/(2*w)));

p = 1.6
w = 0.83
cos = np.maximum(np.cos(w*(tt-c)),0)
fit = a*cos**p * (np.abs(tt-c)<=np.pi/(2*w))
#fit = np.real(fit)
plt.plot(tt,fit);

In [ ]:
claw = stegoton_shapes.setup(num_output_times=200,amp=.5,cells_per_layer=40,width=10,tfinal=800.,
                             solver_type="sharpclaw",shape="cosp",xmax=200)
claw.keep_copy = True

claw.run()

In [ ]:
# amp = 0.5
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(198,210)

w = 0.7
c = 203.31
a = 0.435
plt.plot(tt,a*(np.cos(w*(tt-c)))**2 * (np.abs(tt-c)<=np.pi/(2*w)));

p = 1.6
w = 0.83
cos = np.maximum(np.cos(w*(tt-c)),0)
fit = a*cos**p * (np.abs(tt-c)<=np.pi/(2*w))
#fit = np.real(fit)
plt.plot(tt,fit);

In [ ]:
claw = stegoton_shapes.setup(num_output_times=200,amp=0.6,cells_per_layer=40,width=10,tfinal=800.,
                             solver_type="sharpclaw",shape="cosp",xmax=200)
claw.keep_copy = True

claw.run()

In [ ]:
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(194,202)

w = 0.7
c = 197.72
a = 0.51
plt.plot(tt,a*(np.cos(w*(tt-c)))**2 * (np.abs(tt-c)<=np.pi/(2*w)));

p = 1.
w = 0.99
cos = np.maximum(np.cos(w*(tt-c)),0)
fit = a*cos**p * (np.abs(tt-c)<=np.pi/(2*w))
#fit = np.real(fit)
plt.plot(tt,fit);

In [ ]:
claw = stegoton_shapes.setup(num_output_times=200,amp=0.6,cells_per_layer=40,width=10,tfinal=800.,
                             solver_type="sharpclaw",shape="cos1",xmax=200)
claw.keep_copy = True

claw.run()

In [ ]:
# 40 cells per layer, cosine initialization
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(192,198)

w = 0.7
c = 194.45
a = 0.55
#plt.plot(tt,a*(np.cos(w*(tt-c)))**2 * (np.abs(tt-c)<=np.pi/(2*w)));

p = 1.
w = 1.
cos = np.maximum(np.cos(w*(tt-c)),0)
fit = a*cos**p * (np.abs(tt-c)<=np.pi/(2*w))
plt.plot(tt,fit);

In [ ]:
eps = g[:,1]
K = 1.
sig = np.exp(K*eps)-1
plt.plot(tt,sig)
plt.xlim(192,198)
plt.plot(tt,2./(np.cosh(np.sqrt(2)*(tt-c)))**2, 'k')

In [ ]:
claw = stegoton_shapes.setup(num_output_times=200,amp=0.6,cells_per_layer=160,width=10,tfinal=800.,
                             solver_type="sharpclaw",shape="cos1",xmax=200)
claw.keep_copy = True

claw.run()

In [ ]:
# 160 cells per layer
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(192,198)

w = 0.7
c = 194.40
a = 0.55
#plt.plot(tt,a*(np.cos(w*(tt-c)))**2 * (np.abs(tt-c)<=np.pi/(2*w)));

p = 1.
w = 1.
cos = np.maximum(np.cos(w*(tt-c)),0)
fit = a*cos**p * (np.abs(tt-c)<=np.pi/(2*w))
#fit = np.real(fit)
plt.plot(tt,fit);

In [ ]:
eps = g[:,1]
K = 1.
sig = np.exp(K*eps)-1
plt.plot(tt,sig)
plt.xlim(192,198)
plt.plot(tt,2./(np.cosh(np.sqrt(2)*(tt-194.33)))**2, 'k')

At this point the stegoton has a rather non-smooth shape.  This seems inevitable, since its width is only about 3 layers of the medium, so it "sees" the microstructure of the medium.  But interestingly, it is quite close to exactly $\alpha\cos(t-c)$.

Furthermore, it's traveling almost exactly at speed 1, which is the maximum speed of propagation for small perturbations (note that the peak originally emerges at $x=0$ at $t=3$, so the speed of this wave is about 0.9936).

# Questions

- What happens to this story as we vary $K$ and $\rho$?  For instance if we keep $K_A=\rho_A=1$ and take $K_B=\rho_B$ larger and larger?
- What if we use some other consitutive relation?  E.g. quadratic?
- What happens if we try to generate larger solitary wave solutions of the homogenized equations?  Is the $c_\text{max}$ limit somehow present there?
- Can we prove that a solitary wave can't move faster than $c_\text{max}$?  The idea is that if the wave is smooth then there is some finite region at the front where the amplitude is $<\epsilon$ and the nonlinearity can't overcome the heterogeneous slow-down.

# Extracting the apex

In [ ]:
plt.plot(-claw.frames[-1].q[1,:]/claw.frames[-1].aux[0,:])
plt.xlim(29900,30900)

In [ ]:
q0 = claw.frames[-1].q.copy()
isteg = [30000,30800]
plt.plot(q0[0,isteg[0]:isteg[1]])

In [ ]:
q0[:,:isteg[0]]=0
q0[:,isteg[1]:]=0
plt.plot(q0[0,:])

In [ ]:
np.savetxt('apex_160cpl.txt',q0)

In [ ]:
claw2 = stegoton_shapes.setup(num_output_times=600,cells_per_layer=160,tfinal=30.,
                             solver_type="sharpclaw",xmax=200,ic="./apex_160cpl.txt")
claw2.keep_copy = True

claw2.run()

In [ ]:
q0 = claw2.frames[0].q
#isteg = [30000,30800]
plt.plot(q0[0,:])
plt.xlim(29000,31000)

In [ ]:
q0 = claw2.frames[600].q
#isteg = [30000,30800]
plt.plot(q0[0,:])
#plt.xlim(29000,31000)

In [ ]:
plt.plot(claw.frames[-1].q[0,:])
plt.xlim(7400,7700)

In [ ]:
q0 = claw.frames[-1].q.copy()
isteg = [7450,7700]
plt.plot(q0[0,isteg[0]:isteg[1]])

In [ ]:
q0[:,:isteg[0]]=0
q0[:,isteg[1]:]=0
plt.plot(q0[0,:])

In [ ]:
np.savetxt('apex_40cpl.txt',q0)

In [ ]:
claw2 = stegoton_shapes.setup(num_output_times=200,cells_per_layer=40,tfinal=800.,
                             solver_type="sharpclaw",xmax=200,ic="./apex_40cpl.txt")
claw2.keep_copy = True

claw2.run()

In [ ]:
# 40 cells per layer
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw2.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.xlim(595,601)

w = 0.7
c = 598.36
a = 0.54

p = 1.
w = 1.
cos = np.maximum(np.cos(w*(tt-c)),0)
fit = a*cos**p * (np.abs(tt-c)<=np.pi/(2*w))
plt.plot(tt,fit);

In [ ]:
eps = g[:,1]
K = 1.
sig = np.exp(K*eps)-1
plt.plot(tt,sig)
plt.xlim(595,601)
plt.plot(tt,2./(np.cosh(np.sqrt(2)*(tt-c)))**2, 'k')

In [ ]:
g = np.loadtxt("./_output/_gauges/gauge190.25.txt")
tt = g[:,0]
rhou = g[:,2]
rho = claw2.frames[0].aux[0,15]
u = rhou/rho
plt.plot(tt,-u)
plt.ylim(-0.1,0.1)

In [ ]:
plt.plot(claw2.frames[10].q[0,:])
#plt.xlim(7400,7700)